In [1]:
import os
import string
import h5py

import matplotlib.pyplot as plt
import time, pickle, pandas

import numpy as np

import keras
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import backend as K
import glob
import shutil

import scipy.misc as misc
import scipy.ndimage as im

import tensorflow as tf

%matplotlib inline

Using TensorFlow backend.


In [2]:
batch_size = 10

data_root = '../data/combined/'
train_data_dir = data_root + 'train'
validation_data_dir = data_root + 'test'

train_datagen = ImageDataGenerator(
        shear_range=0.25,
        zoom_range=0.25,
        rotation_range=35,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

batch, y = train_generator.next()
img_height = batch.shape[1]
img_width = batch.shape[2]
channels = batch.shape[3]
num_classes = len(y[0])
print img_height, img_width, channels, num_classes

Found 309 images belonging to 3 classes.
Found 81 images belonging to 3 classes.
256 256 1 3


In [3]:
model = keras.models.load_model('../models/sketchnet4_1/latest.h5')
model.summary()

IOError: Unable to open file (Unable to open file: name = '../models/sketchnet4_1/latest.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
# load latest, add more convs
for i in xrange(4):
    model.pop()
    
model.add(Convolution2D(256, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Convolution2D(512, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

for i, layer in enumerate(model.layers):
    layer.name = 'layer'+str(i)

In [ ]:
opt = rmsprop(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
import utils
reload(utils)

trainer =  utils.ModelTrainer(
                 model,
                 'reference1',
                 opt,
                 train_generator,
                 validation_generator,
                 train_steps=31,
                 val_steps=8)

In [ ]:
trainer.train(20, 0.0001)

In [ ]:
trainer.train(10, 0.00005)

In [ ]:
trainer.train(10, 0.000025)